# Testing functionality of TKM and JAX

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!export XLA_PYTHON_CLIENT_MEM_FRACTION=.1

In [6]:
import jax
# cpu = jax.devices("cpu")[0]
# jax.default_device(cpu)

In [7]:
# from jax.config import config
# config.update("jax_enable_x64", True)

## Load Banana Data

In [8]:
import pandas as pd
from pathlib import Path

In [9]:
data_path = Path("../data/banana.csv")
df_banana = pd.read_csv(data_path)

In [10]:
df_banana.head()

,V1,V2,Class
0,1.140,-0.114,1
1,-1.520,-1.150,2
2,-1.050,0.720,1
3,-0.916,0.397,2
4,-1.090,0.437,2


In [11]:
# expecting labels 1 and -1

# transform label 2 to -1
df_banana.loc[df_banana.Class == 2, 'Class'] = -1
df_banana.head()

,V1,V2,Class
0,1.140,-0.114,1
1,-1.520,-1.150,-1
2,-1.050,0.720,1
3,-0.916,0.397,-1
4,-1.090,0.437,-1


In [12]:
# standardize

# df_banana[['V1','V2']] = (df_banana[['V1','V2']] -  df_banana[['V1','V2']].min() )/ (df_banana[['V1','V2']].max() - df_banana[['V1','V2']].min())
# df_banana.head()

In [13]:
feature_names = ['V1', 'V2']
x = df_banana[feature_names].to_numpy()

label_name = ['Class']
y = df_banana[label_name].to_numpy()

In [14]:
# rescale

x = (x - x.min(axis=0)) / (x.max(axis=0) - x.min(axis=0))

### Data to JAX array

In [15]:
from jax import numpy as jnp
x = jnp.array(x)
y = jnp.array(y)

In [16]:
from jax import random
key = random.PRNGKey(42)

## Tensor Kernel Machine

### Polynomial

In [17]:
from tkm.model import TensorizedKernelMachine as TKM
from jax import jit
from tkm.features import polynomial

model = TKM(M=12,R=6,feature_map=polynomial)

In [18]:
W = model.fit(key,x,y) #,M=12,R=6,feature_map=polynomial)

In [19]:
%timeit model.fit(key,x,y) #,M=12,R=6,feature_map=polynomial)

19.1 ms ± 417 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Predict

In [20]:
from tkm.metrics import accuracy

y_hat = model.predict(x, W) #, feature_map=polynomial, M=12,R=6)
acc = accuracy(y.squeeze(), y_hat)
acc

DeviceArray(0.8918868, dtype=float32)

In [21]:
%timeit model.predict(x, W) #, M=12,R=6, feature_map=polynomial)

1.7 ms ± 7.15 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [25]:
# from jax import jit
# from tkm.model import predict_vmap

y_hat = model.predict_vmap(x, W, feature_map=polynomial, M=12, R=6)
acc = accuracy(y.squeeze(), y_hat)
print(acc)

%timeit model.predict_vmap(x, W, feature_map=polynomial, M=12, R=6)

0.8918868
4.21 ms ± 22 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
from functools import partial

predict_vmap_poly = partial(model.predict_vmap, feature_map=polynomial, M=12, R=6)
predict_vmap_compiled = jit(predict_vmap_poly)

%timeit predict_vmap_compiled(x,W).block_until_ready()

1.7 ms ± 7.71 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


#### fourier features

In [28]:
from tkm.model import TensorizedKernelMachine
from tkm.features import fourier

model_fourier = TensorizedKernelMachine(M=12,R=6,lengthscale=0.5,feature_map=fourier)
W = model_fourier.fit(key,x,y)
y_hat = model_fourier.predict(x, W)
acc = accuracy(y.squeeze(), y_hat)
print(acc)

0.8945283


In [29]:
jax.devices()

[CpuDevice(id=0)]

## batched dotkron

In [31]:
from tkm.model import TensorizedKernelMachine 

model_batch = TensorizedKernelMachine(M=12,R=6,feature_map=polynomial,batch_size=10)
W_batch = model_batch.fit(key,x,y) #,M=12,R=6,feature_map=polynomial)

In [32]:
y_hat = model_batch.predict(x,W_batch)